# Malign

`malign` is a set of experimental methods for sequence alignment that allow:
    
- a unique alphabet for each sequence
- computation of k-best alignments
- usage of assymetric information
- perform single-pass true multiple alignment (instead of combination of pairwise)
- while usable with any type of sequence, including biological, it is intended for linguistic usage mostly
- gaps are full symbols, not "bad"

This notebook explores ideas on how to use it.

In [1]:
import itertools

# Import the library
import malign
from malign import print_alms, print_malms

The most stupid alignment method implemented in `malign` is the "dumb" one. It returns a single alignment for a number of sequences, padding spaces to the left and right without even considering the values of the sequences. It is used for bootstrapping a number of other alignment methods.

In [2]:
# Perform pairwise dumb alignment
seq_a = 'tra'
seq_b = 'fatata'
print_alms(malign.pw_align(seq_a, seq_b, method="dumb"))

# Perform multiwise dumb alignment
seqs = ['tra', 'fra', 'batata', 'virp', 'x']
print_malms(malign.multi_align(seqs, method="dumb"))

A 0 (0.50 / 0.50): - t r a - -
B 0 (0.50 / 0.50): f a t a t a
0 A (0.57) : ['-', 't', 'r', 'a', '-', '-']
0 B (0.57) : ['-', 'f', 'r', 'a', '-', '-']
0 C (0.57) : ['b', 'a', 't', 'a', 't', 'a']
0 D (0.57) : ['-', 'v', 'i', 'r', 'p', '-']
0 E (0.57) : ['-', '-', 'x', '-', '-', '-']


New advanced methods begin with our modified Needleman-Wunsch. The methods work on multidimensional transition matrices that include all symbols and gaps. All correspondences must be provided, even when dealing with the same alphabet.

For illustration, let's start with a common DNA base matrix for pairwise sequence alignment:

  |   | A  |  G |  C |  T | -  |
  |---|----|----|----|----|----|
  | A | 10 | -1 | -3 | -4 | -5 |
  | G | -1 | 7  | -5 | -3 | -5 |
  | C | -3 | -5 | 9  |  0 | -5 |
  | T | -4 | -3 |  0 |  8 |-5  |
  | - | -5 | -5 | -5 | -5 | 0  |

Note that in this case the matrix is symmetric, but this is not necessary as we'll see later. This matrix is available as `malign.utils.DNA_MATRIX`.

Here we start by performing some pairwise alignments on very simple genetic sequences. Note that the parameter `k` allows to return at most k alignments, in terms of best scores, if those are computed. This method is different from normal WN algorithm because it first computes alignments in terms of `seq_a` to `seq_b`, then in terms of `seq_b` to `seq_a`, and takes the mean score as representatitve of the alignment. The results are equal in this case, as the matrix is symmetric.

In [3]:
# Perform pairwise, modifier NW alignment
print("===== Alignment 1")
seq_a = "GATTACA"
seq_b = "A"
print_alms(malign.pw_align(seq_a, seq_b, k=2, method="nw", matrix=malign.utils.DNA_MATRIX))

print("===== Alignment 2")
seq_a = "GATTACA"
seq_b = "ATTT"
print_alms(malign.pw_align(seq_a, seq_b, k=2, method="nw", matrix=malign.utils.DNA_MATRIX))

===== Alignment 1
A 0 (0.00 / -4.00): G A T T A C A
B 0 (-8.00 / -4.00): - - - - - - A
A 1 (0.00 / -5.00): G A T T A C A
B 1 (-10.00 / -5.00): - - - - A - -
===== Alignment 2
A 0 (0.00 / -4.50): G A T T A C A
B 0 (-9.00 / -4.50): - A T T - T -


But what if the matrix is not symmetric? This is not usually the case for genetic data, but let's for sake of experience simulate a condition where sequence B never has a T aligned with a C (from the "point of view" of A, the normal affinities hold). We simulate this by changing the matrix for (C, T), which is now different (not symmmetric) from (T, C), and check the results.

In [4]:
# Perform pairwise, assymetric NW
matrix = malign.utils.DNA_MATRIX.copy()
matrix['C', 'T'] = -99
print_alms(malign.pw_align(seq_a, seq_b, k=4, method="nw", matrix=matrix))

A 0 (-3.00 / -5.50): G A - T T A C A
B 0 (-8.00 / -5.50): - A T T T - - -
A 1 (-3.00 / -5.50): G A T - T A C A
B 1 (-8.00 / -5.50): - A T T T - - -
A 2 (-3.00 / -5.50): G A T T - A C A
B 2 (-8.00 / -5.50): - A T T T - - -
A 3 (-3.00 / -5.50): G A T T A C A -
B 3 (-8.00 / -5.50): - A T T - - - T


You can see that the algorithm now does its best not to align a C in sequence a to a T in sequence B. This can go to extremes, as in the first example below, even though the inverse alignment still work as it commonly does.

In [5]:
print("===== Alignment 1")
print_alms(malign.pw_align("GATTACA", "GATTATA", k=2, method="nw", matrix=matrix))
print("===== Alignment 2")
print_alms(malign.pw_align("GATTATA", "GATTACA", k=2, method="nw", matrix=matrix))

===== Alignment 1
A 0 (-3.00 / -3.00): G A T T A - C A
B 0 (-3.00 / -3.00): G A T T A T - A
A 1 (-3.00 / -3.00): G A T T A C - A
B 1 (-3.00 / -3.00): G A T T A - T A
===== Alignment 2
A 0 (0.00 / 0.00): G A T T A T A
B 0 (0.00 / 0.00): G A T T A C A


The assymetry starts to make more sense when we start dealing with different alphabets, or at least with sequences that indeed from different domains. Take the example below, manually crafted, where we prepare a matrix to align the orthography of Italian and Russian cognates, both written with the standard orthographies. Note how the matrix is assymetric even in terms of gaps: there are some cases where a gap is more or less likely in the correspondence -- sometimes they are even higher than the correspondence to other graphemes.

  |   | а  |  т |  о |  м | Я  | к  | в  | -  |
  |---|----|----|----|----|----|----|----|----|
  | a | 10 | -4 |  3 | -3 | 5  | -4 | -4 | -1 |
  | t | -4 | 10 | -4 | -3 | -4 | -1 | -3 | -3 |
  | o |  4 | -4 | 10 | -5 | 2  | -4 | -2 | 0  |
  | m | -2 | -1 | -4 |  9 |-3  | -2 | 2  | 1  |
  | G | -3 | 1  | -3 | -1 | 4  | 3  | -2 | 2  |
  | i | 5  | -3 | 4  | 0  | 6  | -3 | -3 | 2  |
  | c | -4 | -1 | -5 | -4 | -5 | 4  | 1  | -3 |
  | - | 1  | -2 | -1 | 2  | 2  | -4 | 2  | 0  |

In [6]:
ita_rus = malign.ScoringMatrix(filename="ita_rus.matrix")

print("===== Alignment 1")
print_alms(malign.pw_align("atomo", "атом", k=2, method="nw", matrix=ita_rus))
print("===== Alignment 2")
print_alms(malign.pw_align("Giacomo", "Яков", k=4, method="nw", matrix=ita_rus))

===== Alignment 1
A 0 (0.00 / -1.50): a t o m o
B 0 (-3.00 / -1.50): а т о м -
===== Alignment 2
A 0 (0.00 / -4.50): G i a c o m o
B 0 (-9.00 / -4.50): - Я - к о в -


We can combine more domains for a multiple NW alignment; unlike most multiple NW methods, we don't build a tree internally because an evolutionary perspective does not necessarily apply in our cases (even in face of a unrooted tree); details are provided in the paper. Here we extend our matrix a bit with Greek cognates. Note that, for showcasing, we are only setting the actual correspondences, guiding the alignment a bit too much, and letting the algorithm fill the missing parts. Note that we add correspondences to two graphemes in Italian, "v", "s" which were not used in the example above.

Note that we start with sparse matrix without much information -- one Italian letter, `G`, has no single information.

The more balenced the information is in terms of strong and low association, the better it will usually be.

  |   | -   |  Ι |  α |  β | κ  | μ   | ο  | ς  | τ  | ω  |
  |---|-----|----|----|----|----|-----|----|----|----|----|
  | - |     | -3 |    |    | -7 | -10 |    | -3 |    |    |
  | G |     |    |    |    |    |     |    |    |    |    |
  | a | -7  | 2  | 10 |    |    |     | 6  |    |    | 4  |
  | c |     |    |    |    |  8 |     |    |    | 2  |    |
  | i | -3  | 7  | 5  |    |    |     | 3  |    |    |    |
  | m |     |    |    | 4  |    | 10  |    |    |    |    |
  | o | -7  |    | 4  |    |    |     | 10 |    |    | 10 |
  | s |     |    |    |    |    |     |    |  6 |    |    |
  | t | -10 |    |    |    |  2 |     |    |    | 10 |    |
  | v |     |    |    | 5  |    |     |    |    |    |    |

In [7]:
ita_grk = malign.ScoringMatrix(filename="ita_grk.matrix")


print("===== Alignment 1")
print_alms(malign.pw_align("atomo", "ατομο", k=2, method="nw", matrix=ita_grk))
print("===== Alignment 2")
print_alms(malign.pw_align("Giacomo", "Ιακωβος", k=4, method="nw", matrix=ita_grk))


# Combine the two matrices into a single one, add some points, show a couple of examples
full_matrix = malign.utils.combine_matrices(ita_rus, ita_grk)
full_matrix['o', 'в', 'ο'] = -4
full_matrix['i', '-', 'Ι'] = -4
full_matrix['c', 'к', 'κ'] = 10
for key in [('-', 'к', 'ο'), ('i', 'а', 'Ι'), ('m', 'в', 'β')]:
    print(key, full_matrix[key])

# Do maligns
print("===== Alignment 1")
seqs = ['atomo', "атом", "ατομο"]
print_malms(malign.multi_align(seqs, method="nw", k=4, matrix=full_matrix))

print("===== Alignment 2")
seqs = ["Giacomo", "Яков", "Ιακωβος"]
print_malms(malign.multi_align(seqs, method="nw", k=4, matrix=full_matrix))

===== Alignment 1
A 0 (0.00 / 0.00): a t o m o
B 0 (0.00 / 0.00): α τ ο μ ο
===== Alignment 2
A 0 (-3.00 / -3.00): G i a c o m o -
B 0 (-3.00 / -3.00): - Ι α κ ω β ο ς
('-', '-', 'Ι') [-3, 0.0]
('-', '-', 'α') [0.8666666666666667, 0.0]
('-', '-', 'β') [-0.04999999999999982, 0.0]
('-', '-', 'κ') [-7, 0.0]
('-', '-', 'μ') [-10, 0.0]
('-', '-', 'ο') [0.8666666666666667, 0.0]
('-', '-', 'ς') [-3, 0.0]
('-', '-', 'τ') [0.7000000000000002, 0.0]
('-', '-', 'ω') [1.2000000000000002, 0.0]
('-', 'Я', '-') [0.0, 2]
('-', 'Я', 'Ι') [-3, 2]
('-', 'Я', 'α') [0.8666666666666667, 2]
('-', 'Я', 'β') [-0.04999999999999982, 2]
('-', 'Я', 'κ') [-7, 2]
('-', 'Я', 'μ') [-10, 2]
('-', 'Я', 'ο') [0.8666666666666667, 2]
('-', 'Я', 'ς') [-3, 2]
('-', 'Я', 'τ') [0.7000000000000002, 2]
('-', 'Я', 'ω') [1.2000000000000002, 2]
('-', 'а', '-') [0.0, 1]
('-', 'а', 'Ι') [-3, 1]
('-', 'а', 'α') [0.8666666666666667, 1]
('-', 'а', 'β') [-0.04999999999999982, 1]
('-', 'а', 'κ') [-7, 1]
('-', 'а', 'μ') [-10, 1]
('-', 'а', 

('m', 'Я', 'τ') [6.5, -3]
('m', 'Я', 'ω') [7.0, -3]
('m', 'а', '-') [0.7999999999999998, -2]
('m', 'а', 'Ι') [4.5, -2]
('m', 'а', 'α') [6.666666666666666, -2]
('m', 'а', 'β') [4, -2]
('m', 'а', 'κ') [4.0, -2]
('m', 'а', 'μ') [10, -2]
('m', 'а', 'ο') [6.666666666666666, -2]
('m', 'а', 'ς') [4.25, -2]
('m', 'а', 'τ') [6.5, -2]
('m', 'а', 'ω') [7.0, -2]
('m', 'в', '-') [0.7999999999999998, 2]
('m', 'в', 'Ι') [4.5, 2]
('m', 'в', 'α') [6.666666666666666, 2]
('m', 'в', 'β') [4, 2]
('m', 'в', 'κ') [4.0, 2]
('m', 'в', 'μ') [10, 2]
('m', 'в', 'ο') [6.666666666666666, 2]
('m', 'в', 'ς') [4.25, 2]
('m', 'в', 'τ') [6.5, 2]
('m', 'в', 'ω') [7.0, 2]
('m', 'к', '-') [0.7999999999999998, -2]
('m', 'к', 'Ι') [4.5, -2]
('m', 'к', 'α') [6.666666666666666, -2]
('m', 'к', 'β') [4, -2]
('m', 'к', 'κ') [4.0, -2]
('m', 'к', 'μ') [10, -2]
('m', 'к', 'ο') [6.666666666666666, -2]
('m', 'к', 'ς') [4.25, -2]
('m', 'к', 'τ') [6.5, -2]
('m', 'к', 'ω') [7.0, -2]
('m', 'м', '-') [0.7999999999999998, 9]
('m', 'м', 'Ι')

We can experiment with the canonical example from List (2014)

In [8]:
seqs = ["VOLDEMORT", "WALDEMAR", "VLADIMIR", "VOLODYMIR"]

voldemort_matrix = malign.utils.identity_matrix(seqs, match=2, gap=-2)

In [9]:
print_malms(malign.multi_align(seqs, method="nw", k=4, matrix=voldemort_matrix))

0 A (3.00) : ('V', 'O', 'L', '-', 'D', 'E', 'M', 'O', 'R', 'T')
0 B (3.00) : ('W', 'A', 'L', '-', 'D', 'E', 'M', 'A', 'R', '-')
0 C (3.00) : ('V', '-', 'L', 'A', 'D', 'I', 'M', 'I', 'R', '-')
0 D (3.00) : ('V', 'O', 'L', 'O', 'D', 'Y', 'M', 'I', 'R', '-')


The NW algorithm, or better, the implementation in `malign`, cannot guarantee a minimum number of k-best paths, and is subject to the same kind of issues of normal NW algorithms, such as differences in global and local alignments. One new experimental method offered by the library, that also allows true multisequence alignment, uses graph theory and Yen's algorithm in order to look for alignments. The same matrices are used for both methods, with all the common caveats, such as aligning gaps to states.

In [10]:
# Do maligns
print("===== Alignment 1")
seqs = ['atomo', "атом", "ατομο"]
print_malms(malign.multi_align(seqs, method="yenksp", k=2, matrix=full_matrix))

print("===== Alignment 2")
seqs = ["Giacomo", "Яков", "Ιακωβος"]
print_malms(malign.multi_align(seqs, method="yenksp", k=2, matrix=full_matrix))

print("===== Alignment 3")
seqs = ["VOLDEMORT", "WALDEMAR", "VLADIMIR", "VOLODYMIR"]
print_malms(malign.multi_align(seqs, method="yenksp", k=2, matrix=voldemort_matrix))

===== Alignment 1
0 A (8.90) : ('a', 't', 'o', 'm', 'o')
0 B (8.90) : ('а', 'т', 'о', 'м', '-')
0 C (8.90) : ('α', 'τ', 'ο', 'μ', 'ο')
===== Alignment 2
0 A (3.40) : ('G', 'i', 'a', 'c', 'o', 'm', '-', 'o')
0 B (3.40) : ('-', 'Я', '-', 'к', '-', '-', 'о', 'в')
0 C (3.40) : ('-', 'Ι', 'α', 'κ', 'ω', 'β', 'ο', 'ς')
1 A (3.08) : ('G', 'i', 'a', 'c', 'o', 'm', '-', 'o')
1 B (3.08) : ('Я', '-', '-', 'к', 'о', 'в', '-', '-')
1 C (3.08) : ('-', 'Ι', 'α', 'κ', 'ω', 'β', 'ο', 'ς')
===== Alignment 3
0 A (2.44) : ('V', 'O', 'L', 'D', 'E', 'M', 'O', 'R', 'T')
0 B (2.44) : ('W', 'A', 'L', '-', 'D', 'E', 'M', 'A', 'R')
0 C (2.44) : ('V', '-', 'L', 'A', 'D', 'I', 'M', 'I', 'R')
0 D (2.44) : ('V', 'O', 'L', 'O', 'D', 'Y', 'M', 'I', 'R')
1 A (2.44) : ('V', 'O', 'L', 'D', 'E', 'M', 'O', 'R', 'T')
1 B (2.44) : ('W', 'A', 'L', 'D', 'E', 'M', 'A', 'R', '-')
1 C (2.44) : ('V', '-', 'L', 'A', 'D', 'I', 'M', 'I', 'R')
1 D (2.44) : ('V', 'O', 'L', 'O', 'D', 'Y', 'M', 'I', 'R')
2 A (2.33) : ('V', 'O', 'L', 'D',

One immediate thing to notice is, as with any alignment method, the need for a scorer, which can be harder for our methods given the multidimensional, asymmetric, and non-square matrices. `malign` offers a number of facilities to compute and refine matrices, including some unsupervised learning that tries to maximize the alignment scores (and which generates a preliminary matrix that can be refined by the user).

We have been using them so far, let's investigate in more detail. Matrices will always try to fill missing point by learning from the ones provided. There are two main ways to provide seed data: either as a dictionary with full alignment sites, or as a submatrices (which can include full alignment sites). Let's start with the simplest, providing a full alignment site. We can initialize the matrix and check how both the data that was provided and that which was not are there (the latter being automatically filled in)

In [11]:
s = {
        ("a", "A", "1"): -1,
        ("a", "A", "2"): 4,
        ("a", "A", "3"): 3,
        ("a", "B", "1"): 1,
        ("b", "A", "1"): -10,
        ("b", "A", "2"): 10,
        ("b", "A", "3"): 10,
        ("b", "A", "4"): 10,
        ("c", "B", "1"): 2,
        ("c", "B", "2"): 2,
        ("a", "-", "-"): -2,
        ("b", "-", "-"): -2,
        ("-", "A", "-"): -20,
        ("-", "B", "-"): -20,
        ("-", "-", "1"): -3,
        ("a", "B", "-"): -10,
        ("-", "A", "1"): -100,
        ("-", "A", "2"): -10,
        ("-", "B", "3"): -5,
    }
m = malign.ScoringMatrix(s)
print(m["a", "A", "2"]) # provided
print(m["-", "-", "3"]) # inferred in first round
print(m["c", "-", "4"]) # inferred in second round

('-', '-', '2') [-10, -3]
('-', '-', '3') [-5, -3]
('-', '-', '4') [-3]
('-', 'A', '3') [3, 10, -5, -20, -100, -10]
('-', 'A', '4') [10, -20, -100, -10]
('-', 'B', '1') [1, 2, -3, -100, -20, -5]
('-', 'B', '2') [2, -10, -20, -5]
('-', 'B', '4') [-20, -5]
('a', '-', '1') [-3, -1, 1, -2]
('a', '-', '2') [4, -2]
('a', '-', '3') [3, -2]
('a', '-', '4') [-2]
('a', 'A', '-') [-20, -2, -10, -1, 4, 3]
('a', 'A', '4') [10, -1, 4, 3]
('a', 'B', '2') [2, 4, 1, -10]
('a', 'B', '3') [-5, 3, 1, -10]
('a', 'B', '4') [1, -10]
('b', '-', '1') [-3, -10, -2]
('b', '-', '2') [10, -2]
('b', '-', '3') [10, -2]
('b', '-', '4') [10, -2]
('b', 'A', '-') [-20, -2, -10, 10, 10, 10]
('b', 'B', '-') [-20, -10, -2]
('b', 'B', '1') [1, 2, -10]
('b', 'B', '2') [2, 10]
('b', 'B', '3') [-5, 10]
('b', 'B', '4') [10]
('c', '-', '-') [-2, -2]
('c', '-', '1') [-3, 2]
('c', '-', '2') [2]
('c', 'A', '-') [-20]
('c', 'A', '1') [-1, -10, -100, 2]
('c', 'A', '2') [4, 10, -10, 2]
('c', 'A', '3') [3, 10]
('c', 'A', '4') [10]
('c'

We can now query submatrices, that in this case were inferred as well. In fact, as the product of possible domains is extremely large but usually we are only going to use `n` (for the number of domains), these are inferred on-demand and cached for reuse if necessary (using parameters for inferring that are set when instantiating the matrix).

In [12]:
print(m["a", "A", None])
print(m["a", None, "3"])
print(m[None, "B", "1"])

4
3
2


The alternative, as we already did, is to initialize the scoring matrix with submatrices, which can include corretion points for full domain. We do this by creating full matrices for the sub-domains and providing them when instantiating. Note how the Itlaian Greek matrix here as far less information and we are not providing a Russian/Greek scorer (even though we could).

In [13]:
ita_rus = {("a", "а"):10,
("a", "т"):-4,
("a", "о"):3,
("a", "м"):-3,
("a", "Я"):5,
("a", "к"):-4,
("a", "в"):-4,
("a", "-"):-1,

("t", "а"):-4,
("t", "т"):10,
("t", "о"):-4,
("t", "м"):-3,
("t", "Я"):-4,
("t", "к"):-1,
("t", "в"):-3,
("t", "-"):-3,

("o", "а"):4,
("o", "т"):-4,
("o", "о"):10,
("o", "м"):-5,
("o", "Я"):2,
("o", "к"):-4,
("o", "в"):-2,
("o", "-"):0,

("m", "а"):-2,
("m", "т"):-1,
("m", "о"):-4,
("m", "м"):9,
("m", "Я"):-3,
("m", "к"):-2,
("m", "в"):2,
("m", "-"):1,

("G", "а"):-3,
("G", "т"):1,
("G", "о"):-3,
("G", "м"):-1,
("G", "Я"):4,
("G", "к"):3,
("G", "в"):-2,
("G", "-"):2,

("i", "а"):5,
("i", "т"):-3,
("i", "о"):4,
("i", "м"):0,
("i", "Я"):6,
("i", "к"):-3,
("i", "в"):-3,
("i", "-"):2,

("c", "а"):-4,
("c", "т"):-1,
("c", "о"):-5,
("c", "м"):-4,
("c", "Я"):-5,
("c", "к"):4,
("c", "в"):1,
("c", "-"):-3,

("-", "а"):1,
("-", "т"):-2,
("-", "о"):-1,
("-", "м"):2,
("-", "Я"):2,
("-", "к"):-4,
("-", "в"):2,
("-", "-"):0,}

ita_grk = {    
  ("a", "α"):10,
  ("a", "-"):-5,
  ("-", "α"):-5,
  ("o", "α"):4,
  ("i", "α"):5,
  ("t", "τ"):10,
  ("c", "τ"):2,
  ("a", "ο"):6,
  ("o", "ο"):10,
  ("o", "-"):-10,
  ("-", "ο"):10,
  ("m", "μ"):10,
  ("a", "Ι"):2,
  ("i", "Ι"):7,
  ("t", "κ"):2,
  ("c", "κ"):8,
  ("a", "ω"):4,
  ("o", "ω"):10,
  ("m", "β"):4,
  ("v", "β"):5,
  ("s", "ς"):6,
}
    

ita_rus_m = malign.ScoringMatrix(ita_rus)
ita_grk_m = malign.ScoringMatrix(ita_grk)

print(ita_rus_m["c", "Я"]) # provided
print(ita_grk_m["t", "μ"]) # inferred

# we provide a single point ita/rus/grk
scorer = {("t", "т", "τ"):10}
sub_matrices = {
    (0, 1): ita_rus_m,
    (0, 2): ita_grk_m,
}
irg_m = malign.ScoringMatrix(scorer, sub_matrices=sub_matrices)

print(irg_m["c", "Я", None]) # provided
print(irg_m["t", None, "μ"]) # provided
print(irg_m[None, "Я", "μ"]) # inferred
print(irg_m["a", "а", "α"]) # inferred
print(irg_m["a", "а", "-"]) # inferred
print(irg_m["a", "Я", "μ"]) # inferred

irg_m.save("ira_rus_grk.matrix")

m2 = malign.ScoringMatrix(filename="ira_rus_grk.matrix")
print(m2["c", "Я", None]) # provided
print(m2["t", None, "μ"]) # provided
print(m2[None, "Я", "μ"]) # inferred
print(m2["a", "а", "α"]) # inferred
print(m2["a", "а", "-"]) # inferred
print(m2["a", "Я", "μ"]) # inferred

m2['a', "Я", None]

('-', 'Ι') [2, 7, -5, 10]
('-', 'β') [4, 5, -5, 10]
('-', 'κ') [2, 8, -5, 10]
('-', 'μ') [10, -5, 10]
('-', 'ς') [6, -5, 10]
('-', 'τ') [10, 2, -5, 10]
('-', 'ω') [4, 10, -5, 10]
('a', 'β') [4, 5, 10, -5, 6, 2, 4]
('a', 'κ') [2, 8, 10, -5, 6, 2, 4]
('a', 'μ') [10, 10, -5, 6, 2, 4]
('a', 'ς') [6, 10, -5, 6, 2, 4]
('a', 'τ') [10, 2, 10, -5, 6, 2, 4]
('c', '-') [-5, -10, 2, 8]
('c', 'Ι') [2, 7, 2, 8]
('c', 'α') [10, -5, 4, 5, 2, 8]
('c', 'β') [4, 5, 2, 8]
('c', 'μ') [10, 2, 8]
('c', 'ο') [6, 10, 10, 2, 8]
('c', 'ς') [6, 2, 8]
('c', 'ω') [4, 10, 2, 8]
('i', '-') [-5, -10, 5, 7]
('i', 'β') [4, 5, 5, 7]
('i', 'κ') [2, 8, 5, 7]
('i', 'μ') [10, 5, 7]
('i', 'ο') [6, 10, 10, 5, 7]
('i', 'ς') [6, 5, 7]
('i', 'τ') [10, 2, 5, 7]
('i', 'ω') [4, 10, 5, 7]
('m', '-') [-5, -10, 10, 4]
('m', 'Ι') [2, 7, 10, 4]
('m', 'α') [10, -5, 4, 5, 10, 4]
('m', 'κ') [2, 8, 10, 4]
('m', 'ο') [6, 10, 10, 10, 4]
('m', 'ς') [6, 10, 4]
('m', 'τ') [10, 2, 10, 4]
('m', 'ω') [4, 10, 10, 4]
('o', 'Ι') [2, 7, 4, 10, -10, 10]


('m', 'м', 'κ') [6.0, 9]
('m', 'м', 'μ') [10, 9]
('m', 'м', 'ο') [10.0, 9]
('m', 'м', 'ς') [6.0, 9]
('m', 'м', 'τ') [7.0, 9]
('m', 'м', 'ω') [7.0, 9]
('m', 'о', '-') [-0.5, -4]
('m', 'о', 'Ι') [5.5, -4]
('m', 'о', 'α') [4.5, -4]
('m', 'о', 'β') [4, -4]
('m', 'о', 'κ') [6.0, -4]
('m', 'о', 'μ') [10, -4]
('m', 'о', 'ο') [10.0, -4]
('m', 'о', 'ς') [6.0, -4]
('m', 'о', 'τ') [7.0, -4]
('m', 'о', 'ω') [7.0, -4]
('m', 'т', '-') [-0.5, -1]
('m', 'т', 'Ι') [5.5, -1]
('m', 'т', 'α') [4.5, -1]
('m', 'т', 'β') [4, -1]
('m', 'т', 'κ') [6.0, -1]
('m', 'т', 'μ') [10, -1]
('m', 'т', 'ο') [10.0, -1]
('m', 'т', 'ς') [6.0, -1]
('m', 'т', 'τ') [10, 7.0, -1]
('m', 'т', 'ω') [7.0, -1]
('o', '-', '-') [-10, 0]
('o', '-', 'Ι') [5.5, 0]
('o', '-', 'α') [4, 0]
('o', '-', 'β') [4.5, 0]
('o', '-', 'κ') [6.0, 0]
('o', '-', 'μ') [10.0, 0]
('o', '-', 'ο') [10, 0]
('o', '-', 'ς') [6.0, 0]
('o', '-', 'τ') [7.0, 0]
('o', '-', 'ω') [10, 0]
('o', 'Я', '-') [-10, 2]
('o', 'Я', 'Ι') [5.5, 2]
('o', 'Я', 'α') [4, 2]
('o', 'Я

5

## Scoring Matrices

Matrices in `malign` are ultimately just dictionaries that provide a score for alignment vectors, following an ordered list of domains (or alphabets), which are sets of symbols expressed by strings of at least one character (including the gap symbol, which for must purposes is a symbol like all others). Alignment sites can be full of partial, allowing to score sub-alignments; partial alignment vectors have the same length of the full vectors, with the unrelates sites marked by `None`. Partial alignment vectors must have at least two valid elements (and in most cases will, indeed, have only two, as used for pairwise alignments).

There is not much to decide on the scores themselves, as their organization is free. By definition, the alignment of full gap vectors (that is, vectors where all non-None sites are gaps) is zero, and there is a general tendency that positively correlated sites have positive numbers and negatively correlated sites have negative numbers, but this is not mandatory. In most applications gaps are usually aligned with lower scores, but this is not necessary and, in fact, it is one of the reasons `malign` was developed.

A scoring matrix might be complete or non-complete. Complete matrices are necessary for most operations and can generally be just Python dictionaries; for other cases, the `ScoringMatrix` object helps with the inference of missing data, allowing to complete a matrix.

Here we create one matrix of domain two and print it. By default, the first domain is printed on the vertical.

In [14]:
# Define a scoring matrix for a simple, two domain system
# alphabet a is {'-', 'a', 'b', 'c'}, alphabet b is {'-', 'X', 'Y'}
vectors = {
    ('-', '-') :  0.0,
    ('-', 'X') : -3.0,
    ('-', 'Y') : -9.0,
    ('a', '-') : -3.0,
    ('a', 'X') :  0.0,
    ('a', 'Y') :  8.0,
    ('b', '-') : -5.0,
    ('b', 'X') :  4.0,
    ('b', 'Y') :  4.0,
    ('c', '-') :  2.0,
    ('c', 'X') : -1.0,
    ('c', 'Y') :  7.0,
}

matrix = malign.ScoringMatrix(vectors)
print(matrix.tabulate())

|    |   - |   X |   Y |
|----|-----|-----|-----|
| -  |   0 |  -3 |  -9 |
| a  |  -3 |   0 |   8 |
| b  |  -5 |   4 |   4 |
| c  |   2 |  -1 |   7 |


If some vector is missing, the `ScoringMatrix` will infer it when initialized (for complete vectors) or when necessary with cache (for incomplete vectors). Different methods are offered for such inferece.

In [15]:
v = vectors.copy()
v.pop(('a', '-'))
v.pop(('c', 'Y'))
matrix = malign.ScoringMatrix(v)
print(matrix.tabulate())

('a', '-') [-5.0, 2.0, 0.0, 8.0]
('c', 'Y') [-9.0, 8.0, 4.0, 2.0, -1.0]
|    |   - |   X |   Y |
|----|-----|-----|-----|
| -  |   0 |  -3 |  -9 |
| a  |   1 |   0 |   8 |
| b  |  -5 |   4 |   4 |
| c  |   2 |  -1 |   2 |


As seen above, we can have partial vectors, and a matrix can be initialized with them as well.

In [16]:
# Define a scoring matrix for a simple, three domain system
# alphabet a is {'-', 'a', 'b', 'c'}, alphabet b is {'-', 'X', 'Y'}, alphabet c is {'-', 'i', 'j'}
# note that we already comment out some entries to make gaps
vectors = {
    ('-', '-', '-') :  0.0,
#    ('-', '-', 'i') : -4.0,
    ("-", "-", "j") : -8.0,

    ('-', 'X', '-') : -5.0,
    ('-', 'Y', '-') : -5.0,
    ('a', '-', '-') :  0.0,
#    ('a', 'X', '-') :  0.0,
    ('a', 'Y', '-') :  8.0,
    ('b', '-', '-') :  0.0,
    ('b', 'X', '-') :  4.0,
    ('b', 'Y', '-') :  4.0,
    ('c', '-', '-') :  0.0,
    ('c', 'X', '-') : -1.0,
    ('c', 'Y', '-') : -7.0,    
    
    ('-', 'X', 'i') : -3.0,
    ('-', 'Y', 'i') : -9.0,
    ('a', '-', 'i') : -3.0,
#    ('a', 'X', 'i') :  0.0,
#    ('a', 'Y', 'i') :  8.0,
    ('b', '-', 'i') : -5.0,
    ('b', 'X', 'i') :  4.0,
    ('b', 'Y', 'i') :  4.0,
    ('c', '-', 'i') :  2.0,
    ('c', 'X', 'i') : -1.0,
    ('c', 'Y', 'i') :  7.0,
    
    ('-', 'X', 'j') : -5.0,
    ('-', 'Y', 'j') : -6.0,
    ('a', '-', 'j') :  3.0,
    ('a', 'X', 'j') :  8.0,
    ('a', 'Y', 'j') :  8.0,
    ('b', '-', 'j') : -6.0,
    ('b', 'X', 'j') :  5.0,
    ('b', 'Y', 'j') :  4.0,
    ('c', '-', 'j') : -5.0,
#    ('c', 'X', 'j') :  6.0,
    ('c', 'Y', 'j') :  7.0,
}
matrix = malign.ScoringMatrix(vectors)
print(matrix.tabulate())

('-', '-', 'i') [-3.0, -5.0, 2.0, -3.0, -9.0, -8.0]
('a', 'X', '-') [-5.0, 4.0, -1.0, 0.0, 8.0, 8.0]
('a', 'X', 'i') [-3.0, 4.0, -1.0, -3.0, 8.0]
('a', 'Y', 'i') [-9.0, 4.0, 7.0, -3.0, 8.0, 8.0]
('c', 'X', 'j') [-5.0, 8.0, 5.0, -5.0, 7.0, -1.0, -1.0]
|    |   /-/- |   /-/i |   /-/j |   /X/- |   /X/i |   /X/j |   /Y/- |   /Y/i |   /Y/j |
|----|--------|--------|--------|--------|--------|--------|--------|--------|--------|
| -  |      0 |     -4 |     -8 |     -5 |     -3 |     -5 |     -5 |   -9   |     -6 |
| a  |      0 |     -3 |      3 |      2 |     -1 |      8 |      8 |    5.5 |      8 |
| b  |      0 |     -5 |     -6 |      4 |      4 |      5 |      4 |    4   |      4 |
| c  |      0 |      2 |     -5 |     -1 |     -1 |     -1 |     -7 |    7   |      7 |
